In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from autoplan.generator import ProgramGenerator
from grammars.pythagorean.program import Program, PythagoreanLabels
from autoplan.dataset import build_synthetic_dataset
from autoplan.trainer import ParserTrainer
from autoplan.parsing import Parser
from autoplan.token import JavaTokenizer
from tqdm import tqdm_notebook as tqdm
import torch
import pandas as pd

# device = torch.device('cpu')
device = torch.device('cuda:0')

In [9]:
program_grammar = Program()
generator = ProgramGenerator(grammar=program_grammar)
program, choices, choice_options = generator.generate()
print(program)
print(choices)
print(choice_options)


            public class PythagoreanTheorem extends ConsoleProgram {
                public void run() {
                    
        println("Enter values to compute the Pythagorean Theorem.");
        int a = readInt("a:");
        int b = readInt("b:");
        int c = Math.sqrt(Math.pow(a, 2) + Math.pow(b, 2));
        println("c:" + c);
        
                }
            }
        
[('START', 0), ('naming_scheme', 0), ('num_type', 0), ('print_function', 0), ('main_prompt', 0), ('input_prompt', 0), ('uses_method', 0), ('output_prompt', 1)]
{'START': [(1.0, None)], 'naming_scheme': [(0.5, 'a'), (0.5, 'x')], 'num_type': [(0.5, 'int'), (0.5, 'double')], 'print_function': [(0.5, 'println'), (0.5, 'print')], 'main_prompt': [(0.699999988079071, 'Enter values to compute the Pythagorean Theorem.'), (0.15000000596046448, 'This program finds the hypotenuse, C, of a triangle with sides A and B.'), (0.15000000596046448, 'This program runs the Pythagorean Theorem. Choose values a and b.')]

In [10]:
dataset = build_synthetic_dataset(
    N_train=1000, 
    N_val=100, 
    tokenizer=JavaTokenizer(),
    grammar=program_grammar)

In [11]:
batch = next(iter(dataset.loader(dataset.val_dataset)))

In [14]:
trainer = ParserTrainer(program_grammar, dataset, device)

In [ ]:
losses = []
accuracy = []
for _ in tqdm(range(20)):
    losses.append(trainer.train_one_epoch())
    accuracy.append(trainer.eval())
pd.Series(losses).plot()